In [0]:
# Lab1 Regression实现

In [0]:
import sys
import pandas as pd
import numpy as np
from google.colab import drive

In [0]:
# 读取数据
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/hw1/train.csv', encoding='big5')

# **Extract Features (1)**
![圖片說明](https://drive.google.com/uc?id=1LyaqD4ojX07oe5oDzPO99l9ts5NRyArH)
![圖片說明](https://drive.google.com/uc?id=1ZroBarcnlsr85gibeqEF-MtY13xJTG47)

將原始 4320 * 18 的資料依照每個月分重組成 12 個 18 (features) * 480 (hours) 的資料。 

In [0]:
#data = data.iloc
data = data.iloc[:,3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
# 预处理数据。源数据有12个月，每个月有20天，每天有24个小时，每小时有18个特征，
# 故预处理数据为12 * 18 * 480的矩阵
month_data = {}
for month in range(12):
  tmp_data = np.empty([18,480])
  for day in range(20):
    tmp_data[:,day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
  month_data[month] = tmp_data

# **Extract Features (2)**
![alt text](https://drive.google.com/uc?id=1wKoPuaRHoX682LMiBgIoOP4PDyNKsJLK)
![alt text](https://drive.google.com/uc?id=1FRWWiXQ-Qh0i9tyx0LiugHYF_xDdkhLN)

每個月會有 480hrs，每 9 小時形成一個 data，每個月會有 471 個 data，故總資料數為 471 * 12 筆，而每筆 data 有 9 * 18 的 features (一小時 18 個 features * 9 小時)。

對應的 target 則有 471 * 12 個(第 10 個小時的 PM2.5)

In [28]:
# 提取特征2：
# 因为要根据九个小时推断第十个小时，故需要将数据按每十个小时形成一段data，其中前九个作为输入，后一个作为输出。分类的方法是0-9，1-10，2-11这样分，故480份数据可分为471个data
# ，而每个data里又有9*18的特征(9是前九个小时，18是每个小时有18个特征),所以对应的target有12 * 471个
x = np.empty([12 * 471, 18 * 9], dtype=float)
y = np.empty([12 * 471, 1], dtype=float)
for month in range(12):
  for day in range(20):
    for hour in range(24):
      if day == 19 and hour > 14:
        continue
      x[month * 471 + day * 24 + hour, :] = month_data[month][:, day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)
      y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9]

print(x)

print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


In [29]:
# 因为每个特征的范围不同，故需要对特征进行标准化操作
mean_x = np.mean(x, axis=0) # numpy.mean()求平均值，axis=0表示对列求
std_x = np.std(x, axis=0) # 求方差
for i in range(len(x)):
  for j in range(len(x[0])):
    if std_x[j] != 0:
      x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

array([[-1.35825331, -1.35883937, -1.359222  , ...,  0.26650729,
         0.2656797 , -1.14082131],
       [-1.35825331, -1.35883937, -1.51819928, ...,  0.26650729,
        -1.13963133, -1.32832904],
       [-1.35825331, -1.51789368, -1.67717656, ..., -1.13923451,
        -1.32700613, -0.85955971],
       ...,
       [-0.88092053, -0.72262212, -0.56433559, ..., -0.57693779,
        -0.29644471, -0.39079039],
       [-0.7218096 , -0.56356781, -0.72331287, ..., -0.29578943,
        -0.39013211, -0.1095288 ],
       [-0.56269867, -0.72262212, -0.88229015, ..., -0.38950555,
        -0.10906991,  0.07797893]])

In [30]:
# 之后将训练数据分割成验证集来验证
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8):, :]
y_validation = y[math.floor(len(y) * 0.8):, :]
print(x_train_set)
print('------------------')
print(y_train_set)
print('------------------')
print(x_validation)
print('------------------')
print(y_validation)
print('------------------')


[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
------------------
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
------------------
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 

In [0]:
# 开始进行Regression进行训练，即y=w1*x1 + w2*x2 + ... + wn * xn + b
# 由于常数项的存在，故需要在原先的维度(18 * 9)上面新增一个维度
dim = 18 * 9 + 1
w = np.zeros([dim, 1]) # 权重应该初始化为0
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1]) # adagrad表达式为w'=w-lr*gd/sqrt(Σgd^2))，为了避免分母为0，加上一个很小的eps
eps = 0.000000000001
for t in range(iter_time):
  loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))) # loss_function=(x*w - y)^2
  gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y)
  adagrad += gradient ** 2
  w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
# 保存训练的权重
np.save('weight.npy', w)
w

In [37]:
# 测试数据集
testdata = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/hw1/test.csv', header=None, encoding='big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18 * 9], dtype=float)
for i in range(240):
  test_x[i, :] = test_data[18 * i : 18 * (i+1), :].reshape(1, -1)
for i in range(len(test_x)):
  for j in range(len(test_x[0])):
    if std_x[j] != 0:
      test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240,1]), test_x), axis=1).astype(float)
test_x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , -0.24447681, -0.24545919, ..., -0.67065391,
        -1.04594393,  0.07797893],
       [ 1.        , -1.35825331, -1.51789368, ...,  0.17279117,
        -0.10906991, -0.48454426],
       [ 1.        ,  1.5057434 ,  1.34508393, ..., -1.32666675,
        -1.04594393, -0.57829812],
       ...,
       [ 1.        ,  0.3919669 ,  0.54981237, ...,  0.26650729,
        -0.20275731,  1.20302531],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  2.98427476,
         3.26367657,  1.76554849]])

In [43]:
# 预测
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

array([[ 5.17496040e+00],
       [ 1.83062143e+01],
       [ 2.04912181e+01],
       [ 1.15239429e+01],
       [ 2.66160568e+01],
       [ 2.05313481e+01],
       [ 2.19065510e+01],
       [ 3.17364687e+01],
       [ 1.33916741e+01],
       [ 6.44564665e+01],
       [ 2.02645688e+01],
       [ 1.53585761e+01],
       [ 6.85894728e+01],
       [ 4.84281137e+01],
       [ 1.87023338e+01],
       [ 1.01885957e+01],
       [ 3.07403629e+01],
       [ 7.11322178e+01],
       [-4.13051739e+00],
       [ 1.82356940e+01],
       [ 3.85789223e+01],
       [ 7.13115197e+01],
       [ 7.41034816e+00],
       [ 1.87179553e+01],
       [ 1.49372503e+01],
       [ 3.67197367e+01],
       [ 1.79616970e+01],
       [ 7.57894629e+01],
       [ 1.23093102e+01],
       [ 5.62953517e+01],
       [ 2.51131609e+01],
       [ 4.61024867e+00],
       [ 2.48377055e+00],
       [ 2.47594223e+01],
       [ 3.04802805e+01],
       [ 3.84639307e+01],
       [ 4.42023106e+01],
       [ 3.00868360e+01],
       [ 4.0

In [44]:
import csv

with open('submit.csv', mode='w', newline='') as submit_file:
  csv_writer = csv.writer(submit_file)
  header = ['id', 'value']
  print(header)
  csv_writer.writerow(header)
  for i in range(240):
    row = ['id_' + str(i), ans_y[i][0]]
    csv_writer.writerow(row)
    print(row)

['id', 'value']
['id_0', 5.174960398984733]
['id_1', 18.30621425352789]
['id_2', 20.491218094180542]
['id_3', 11.523942869805332]
['id_4', 26.616056752306132]
['id_5', 20.53134808176123]
['id_6', 21.90655101879738]
['id_7', 31.73646874706884]
['id_8', 13.391674055111707]
['id_9', 64.45646650291955]
['id_10', 20.26456883615946]
['id_11', 15.35857607736124]
['id_12', 68.58947276926726]
['id_13', 48.42811374745721]
['id_14', 18.702333824193207]
['id_15', 10.188595737466695]
['id_16', 30.74036285982043]
['id_17', 71.13221776355107]
['id_18', -4.130517391262448]
['id_19', 18.235694016428692]
['id_20', 38.578922275007756]
['id_21', 71.31151972531332]
['id_22', 7.410348162634086]
['id_23', 18.717955330321395]
['id_24', 14.937250260084571]
['id_25', 36.71973669470532]
['id_26', 17.961697005662703]
['id_27', 75.78946287210537]
['id_28', 12.309310248614473]
['id_29', 56.29535173964961]
['id_30', 25.11316086566149]
['id_31', 4.6102486740940325]
['id_32', 2.4837705545150315]
['id_33', 24.759422261